#  Submit geo opt for molecules, slabs and bulks. GW for molecules

In [ ]:
# General imports.
import nglview
import numpy as np
import ipywidgets as ipw
from IPython.display import clear_output
import itertools

from traitlets import dlink

# AiiDA imports.
%aiida
from aiida.orm import Dict as Dict
from aiida.orm import SinglefileData, Node

# Workchains.
from aiida_cp2k.workchains.base import Cp2kBaseWorkChain
from apps.surfaces.molecules.gw_workchain import GwWorkChain

# AiiDA lab imports.
from aiidalab_widgets_base.utils import string_range_to_list, list_to_string_range
from aiidalab_widgets_base import CodeDropdown, StructureManagerWidget, BasicStructureEditor
from aiidalab_widgets_base import StructureBrowserWidget, StructureUploadWidget, SubmitButtonWidget, SmilesWidget

# Custom imports.
from apps.surfaces.widgets.create_xyz_input_files import make_geom_file
from apps.surfaces.widgets.ANALYZE_structure import StructureAnalyzer
from apps.surfaces.widgets.build_slab import BuildSlab
from apps.surfaces.widgets.cp2k2dict import CP2K2DICT
from apps.surfaces.widgets.inputs import InputDetails
from apps.surfaces.widgets.empa_viewer import EmpaStructureViewer
from apps.surfaces.widgets.import_cdxml import CdxmlUpload2GnrWidget

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Structure selector.
empa_viewer = EmpaStructureViewer()

structure_selector = StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        StructureBrowserWidget(title="AiiDA database"),
        StructureUploadWidget(title="Import from computer"),
        SmilesWidget(title="From SMILES"),
        CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    editors = [
        BasicStructureEditor(title="Edit structure")
    ],
    storable=False, node_class='StructureData')
display(structure_selector)

# Code.
computer_code_dropdown = CodeDropdown(input_plugin='cp2k')
#input_details = InputDetails()
create_input = ipw.Button(description="Create Input")

In [ ]:
#dlink((empa_viewer, 'details'), (input_details, 'details'))
#dlink((computer_code_dropdown, 'selected_code'),(input_details, 'selected_code'))

def get_plain_input_gw():
    error_msg,submit_dict = CP2K2DICT(input_lines = input_details.plain_input.value)
    if error_msg != "":
        print(error_msg)
        return
    
    builder = GwWorkChain.get_builder()
    final_dict = input_details.final_dictionary.copy()
    builder.metadata.description = final_dict['description']
    builder.metadata.label = final_dict['workchain']    
    builder['final_dict'] = Dict(dict=final_dict)

    ## GW CASE USE GW workchian

    
    builder['submit_dict'] = Dict(dict=submit_dict)

    ## create simple SCF input from the GW input
    structure = structure_selector.structure_node
    if final_dict['ic_plane_z']:
        builder['xyz_gw'] = make_geom_file(structure,
                                 Str("mol.xyz"),
                                 spin_u=Str(final_dict['spin_u']),
                                 spin_d=Str(final_dict['spin_d']),
                                 ic_plane_z=Float(final_dict['ic_plane_z'])
                                )
    else:
        builder['xyz_gw'] = make_geom_file(structure,
                                 Str("mol.xyz"),
                                 spin_u=Str(final_dict['spin_u']),
                                 spin_d=Str(final_dict['spin_d'])
                                )        

    builder['code'] = computer_code_dropdown.selected_code

    return builder



In [ ]:
def after_submission(_=None):   
    structure_selector.value = None


btn_submit_gw = SubmitButtonWidget(GwWorkChain, 
                                input_dictionary_function=get_plain_input_gw
                               )

btn_submit_gw.on_submitted(after_submission)

In [ ]:
output = ipw.Output()
def update_all(_=None):
    btn_submit.btn_submit.disabled=True    

def create(_=None):
    if computer_code_dropdown.selected_code is None:
        msg = 'please select a computer'
        can_submit = False
    else:            
        can_submit, msg, details_dict = input_details.return_final_dictionary()
       # if len(details_dict['description']) < 4 :
       #     msg = 'Please provide a better description'
       #     can_submit = False
            
        btn_submit.btn_submit.disabled = not can_submit
    with output:
        clear_output()
        if can_submit:
            print("Ready to submit a ",details_dict['workchain']," type of calculation" )
            if details_dict['workchain'] == 'GWWorkChain':
                display(btn_submit_gw)
            else:
                display(btn_submit)
        else:
            print(msg)

create_input.on_click(create)
input_details.observe(update_all, names='details')

In [ ]:
display(ipw.VBox([computer_code_dropdown]), output)